# Air Quality Problem

This is a modification from Prof. Culver of a problem in Haith, D. A. (1982). Environmental systems optimization.

Burning Eyes County has an air pollution problem. The air pollutant KOF, which is emitted from three major sources in the county, has been declared a hazard to health by the state environmental regulatory agency. Emissions of KOF from the three sources are given in Table 1.

| Source | Emissions (kg/day) | Stack Height (m) | Removal Cost (\$/kg) |
| ------ | ------------------ | ---------------- | -------------------- |
| 1      | 34,560             | 50               | 0.020                |
| 2      | 172,800            | 200              | 0.045                |
| 3      | 103,680            | 30               | 0.060                |

Air pollution limits are set assuming class B stability with a 1.2 m/s mean wind speed at a 10 m high anemometer and slight daytime insolation. Five ground level locations (receptors) in the county have been selected as places where health effects are most important. At each of these locations, an ambient air quality standard for KOF has been set at 150 $\mu g/m^3$. The coordinates of sources and receptors are given in Table 2 (x-y coordinates in km, $x$-axis parallel to wind direction).

| Sources | Receptors |
| ------- | --------- |
| (5, 7)  | (6, 1.5)  |
| (7, 5)  | (8, 7)    |
| (8, 5)  | (10, 3)   |
|         | (12.5, 6) |
|         | (15.5, 5) |

Using the Gaussian plume model and a linear program, determine how much each source should reduce their KOF emissions by to meet the air quality standard at all receptors at the lowest cost.

In [ ]:
!pip install gurobipy
!pip install pynverse

$\color{red}{\text{Replace the license in the code below with your own.}}$

In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.stats as ss
import scipy.integrate as integrate
from pynverse import inversefunc
import matplotlib.pyplot as plt

# create environment with Gurobi academic license
params = {
"WLSACCESSID": '69c0357e-257d-4855-a3f0-446b8e822abd',
"WLSSECRET": '11082e14-04a8-4439-a62d-f3acd3d46920',
"LICENSEID": 2471772,
}
env = gp.Env(params=params)

In class we determined the mathematical formulation of this problem. First, we defined our notation.

Parameters:
* $U^a$ = wind speed at anemometer (m/s)
* $U^h_s$ = wind speed at source $s$ located at height $h$ (m/s)
* $H^a$ = height of anemometer
* $H_s$ = height of smoke stack at source s
* $Q_s$ = current emission rate of pollutant from source $s$ (g/s)
* $cost_s$ = \$/g of pollutant treated at source $s$
* $x_s$ = x-coordinate of source s (km)
* $y_s$ = y-coordinate of source s (km)
* $x_r$ = x-coordinate of receptor r (km)
* $y_r$ = y-coordinate of receptor r (km)
* $\Delta x(s,r)$ = distance along x-axis between source $s$ and receptor $r$ (km)
* $\Delta y(s,r)$ = distance along y-axis between source $s$ and receptor $r$ (m)
* $\sigma_y(s,r)$ = dispersion of pollutant from source $s$ along y-axis at distance $\Delta x(s,r)$ from source (m)
* $\sigma_z(s,r)$ = dispersion of pollutant from source $s$ along z-axis at distance $\Delta x(s,r)$ from source (m)

Decision variables:
* $x_s$ = emission rate of pollutant from source $s$ needed to meet air quality standard

Optimization Problem:
\begin{align}
\text{Minimize} &\quad \sum_{s=1}^{S} cost_s \times (Q_s - x_s)\\
\text{s.t.} &\quad T_{s,r} \times x_s \leq 0.00015 g \quad \forall r \in R\\
& \quad x_s \geq 0 \quad \forall s \in S\\
\text{where} &\\
& T_{s,r} = \frac{\exp\Big(-\Big[\frac{\Delta y(s,r)^2}{2\sigma_y(s,r)^2} + \frac{H_s^2}{2\sigma_z(s,r)^2}\Big]\Big)}{\pi U^h_s \sigma_y(s,r) \sigma_z(s,r)}\\
& U^h_s = U^a \Bigg(\frac{H_s}{H^a}\Bigg)^p\\
& \Delta y(s,r) = |1000 (y_r - y_s) |\\
& \sigma_y(s,r) = a\Delta x(s,r)^{b+c\ln(\Delta x(s,r))}\\
& \sigma_z(s,r) = d\Delta x(s,r)^{e+f\ln(\Delta x(s,r))}\\
& \Delta x(s,r) = \max(0, x_r - x_s)\\
& a,b,c,d,e,f,p \text{ are coefficients defined by the wind class}
\end{align}

The wind class coefficients are defined as follows:  

| Stability Class	| a |	b | c | d | e | f | p | Applicable Range |
| --------------- | - | - | - | - | - | - | - | ---------------- |
| A | 209.6 | 0.8804 | -0.006902 | 310.4 | 1.773 | 0.1879 | 0.15 | 0.1 < x < 0.45 km |
| A | 209.6 | 0.8804 | -0.006902 | 453.9 | 2.117| 0.0	| 0.15 | 0.45 < x < 3.1 km |
| B	| 154.7	| 0.8932 | -0.006271 | 109.8 | 1.064	| 0.01163	| 0.15 | 0.1 < x < 32 km |
| C	| 103.3	| 0.9112 | -0.004845 | 61.14 | 0.9147	| 0.0	| 0.2 | 0.1 < x < 100 km |
| D	| 68.28 |	0.9112 | -0.004845 | 30.38 | 0.7309 | -0.03200 | 0.25 | 0.1 < x < 100 km |
| E	| 51.05 | 0.9112 | -0.004845 | 21.14 | 0.6802	| -0.04522 | 0.4 | 0.1 < x < 100 km |
| F	| 33.96	| 0.9112 | -0.004845 | 13.72 | 0.6584 | -0.05367 | 0.6 | 0.1 < x < 100 km |




First we define the parameters.

In [ ]:
#################### Parameters ####################
# x and y coordinates (km) of sources and receptors
source_coords = np.array([[5,7],
                 [7,5],
                 [8,5]])
nSources = len(source_coords)

receptor_coords = np.array([[6,1.5],
                   [8,7],
                   [10,3],
                   [12.5,6],
                   [15,5]])
nReceptors = len(receptor_coords)

# sigmay, sigmaz and wind speed coefficients for class B
a = 154.7; b = 0.8932; c = -0.00627
d = 109.8; e = 1.064; f = 0.01163
p = 0.15

# heights and wind speeds at sources
H = np.array([50, 200, 30]) # heights of smoke stacks in meters
Ha = 10 # height of anemometer in meters
Ua = 1.2 # wind speed at anemometer in m/s
Uh = Ua*(H/Ha)**p

# source emissions before treatment (kg/day) converted to (g/s)
Q = np.array([34560, 172800, 103680])*1000/(3600*24)

# treatment costs ($/kg) converted to ($/g)
cost = np.array([0.02, 0.045, 0.06])/1000

Next, we compute the transfer coefficients between all sources and receptors.

In [ ]:
# find concentration at each receptor, which cannot exceed 0.00015 g/m^3
T = np.zeros([nSources,nReceptors])
dx = np.zeros([nSources,nReceptors])
dy = np.zeros([nSources,nReceptors])
sigma_y = np.zeros([nSources,nReceptors])
sigma_z = np.zeros([nSources,nReceptors])
for r, receptor in enumerate(receptor_coords):
    for s, source in enumerate(source_coords):
        # x distance in km; is 0 if receptor not downwind of source
        dx[s,r] = max(0.0,receptor[0] - source[0])
        # y distance in m; is from centerline, so take absolute value
        dy[s,r] = np.abs(1000*(receptor[1] - source[1]))
        sigma_y[s,r] = a*dx[s,r]**(b+c*np.log(dx[s,r]))
        sigma_z[s,r] = d*dx[s,r]**(e+f*np.log(dx[s,r]))
        # only calculate transfer coefficient if receptor downwind of cource
        if dx[s,r] > 0:
            T[s,r] = np.exp( -( dy[s,r]**2/(2*sigma_y[s,r]**2) + \
                               H[s]**2/(2*sigma_z[s,r]**2) ) ) / \
                (np.pi*Uh[s]*sigma_y[s,r]*sigma_z[s,r])

Finally, set up and solve the linear programming problem.

In [ ]:
#################### Model ####################
# Create the model within the Gurobi environment
model = gp.Model('Air Quality',env=env)

# define decision variables: x = g/s of pollutant coming from each source
x = model.addVars(3, vtype=GRB.CONTINUOUS)

AQconstrs = model.addConstrs(gp.quicksum(T[s,r]*x[s] for s in range(nSources)) <=
                                         0.00015 for r in range(nReceptors))

# find reduction of KOF, Q-x, to minimize cost of treatment that meets AQ standards
total_cost = gp.quicksum(cost[s] * (Q[s] - x[s]) for s in range(nSources))

model.setObjective(total_cost)
model.write('air_quality.lp')
model.optimize()

Print the results.

In [ ]:
for s in range(nSources):
  print("Pollutant emissions from source %(source)d: %(emission)0.1f g/s" % {"source": s+1, "emission": x[s].x})
  print("Reduction in pollutant emissions from source %(source)d: %(reduction)0.1f g/s" % {"source": s, "reduction": Q[s] - x[s].x})
  print("\n")

We solved this problem based on the mean wind speed. But the mean wind speed won't result in the mean pollutant concentration since the relationship between them is nonlinear. To determine how much we need to reduce emissions from each source by so that the mean pollutant concentration meets the air quality standard, we need to compute the mean transfer coefficient. This can be computed as follows:  
$f(T) = f(U_a(T)) \Big|\frac{\partial U_a}{\partial T}\Big|$.

Below we compute $f(T)$ from this equation assuming $U_a$ follows a 3-parameter log-normal distribution with shape parameter $\sigma$, scale parameter $\exp(\mu)$ and location parameter (lower bound) $\tau$:  
$f(U_a) = \frac{1}{U_a\sqrt{2\pi \sigma^2}}\exp\Big(-\frac{(\ln(U_a-\tau)-\mu)^2}{2\sigma^2}\Big)$

From the pdf $f(T)$, we can also compute its cdf through integration: $F(T) = \int_0^t f(T) dT$,   
its point-percentile function through inversion: $F^{-1}(T)$,    
and its moments through product-moment integration: $E[T^m] = \int_0^{UB} T^m f(T) dT$  
where $UB$ is the value of $T$ when $U_a$ is at its lower bound $\tau$.

In [ ]:
# convert f(Ua) to f(T)
class TransferCoeffDist:
    def __init__(self, mu_Ua, sigma_Ua, tau_Ua, Ha, H, p, dy, sigma_y, sigma_z):
        self.Ha = Ha # height of anemometer
        self.H = H # height of smoke stack
        self.p = p # wind speed coefficient
        self.dy = dy # distance from smoke stack along y axis
        self.sigma_y = sigma_y # y dispersion coefficient
        self.sigma_z = sigma_z # z dispersion coefficient
        self.mu_Ua = mu_Ua # log(scale parameter) of Ua
        self.sigma_Ua = sigma_Ua # shape parameter of Ua
        self.tau_Ua = tau_Ua # location parameter of Ua
        self.UB = (self.H/self.Ha)**(-self.p) * \
                      np.exp(-(self.dy**2/(2*self.sigma_y**2) + \
                                self.H**2/(2*self.sigma_z**2))) / \
                       (np.pi*self.sigma_y*self.sigma_z*tau_Ua) # value of T at Ua = tau_Ua

    def pdf(self, x):
        T = x
        Ua_T = (self.H/self.Ha)**(-self.p) * \
                      np.exp(-(self.dy**2/(2*self.sigma_y**2) + \
                                self.H**2/(2*self.sigma_z**2))) / \
                       (np.pi*self.sigma_y*self.sigma_z*T)
        f_Ua_T = ss.lognorm.pdf(Ua_T, s=self.sigma_Ua, loc=self.tau_Ua, scale=np.exp(self.mu_Ua))
        abs_dUa_dT = np.abs(-Ua_T/T)
        f_T = f_Ua_T * abs_dUa_dT
        return f_T

    def cdf(self, x):
        # integrate pdf from 0 to x
        return integrate.quad(lambda x: self.pdf(x), 0, x)[0]

    def ppf(self, q):
        # invert cdf
        cdf = lambda x: self.cdf(x)
        ppf = inversefunc(cdf, domain=[0,self.UB])
        return ppf(q)

    def moment(self, m):
        # integrate x^m * f(x) from 0 to UB
        return integrate.quad(lambda x: (x**m * self.pdf(x)), 0, self.UB)[0]

    def central_moment(self, m):
        # integrate (x-mu)^m * f(x) from 0 to UB
        mu = self.moment(1)
        return integrate.quad(lambda x: ((x-mu)**m * self.pdf(x)), 0, self.UB)[0]

Below is an example of how to initialize a variable with this distribution and calculate its pdf, cdf, ppf, and moments.

In [ ]:
mu_Ua = 0.1 # example, not the value for the homework
sigma_Ua = 0.5 # example, not the value for the homework
tau_Ua = 0.01 # same as for the homework
s = 1 # example with second source
r = 3 # example with fourth receptor

# initialize object of TransferCoeffDist class
Tdist_example = TransferCoeffDist(mu_Ua, sigma_Ua, tau_Ua, Ha, H[s], p, dy[s,r], sigma_y[s,r], sigma_z[s,r])

UB_T = Tdist_example.UB # find the upper bound of the distribution of T
mean_T = Tdist_example.moment(1) # find the mean of the distribution of T
var_T = Tdist_example.central_moment(2) # find the variance of the distribution of T
f_mean_T = Tdist_example.pdf(mean_T) # find the value of the pdf at the mean
F_mean_T = Tdist_example.cdf(mean_T) # find the value of the cdf at the mean
q_95_T = Tdist_example.ppf(0.95) # find the 95th percentile of the distribution of T

print("Upper Bound of T: ", UB_T)
print("Mean of T: ", mean_T)
print("Variance of T: ", var_T)
print("f(T=meanT): ", f_mean_T)
print("F(T=meanT): ", F_mean_T)
print("95th percentile of T: ", q_95_T)

Plot the pdf and cdf of T.

In [ ]:
t = np.arange(0,UB_T,UB_T/100)
pdf_t = Tdist_example.pdf(t)
cdf_t = np.empty([len(pdf_t)])
for i in range(len(t)):
  cdf_t[i] = Tdist_example.cdf(t[i])

fig, ax = plt.subplots(2,1)
ax[0].loglog(t,pdf_t)
ax[0].set_ylabel("f(T)")
ax[1].semilogx(t,cdf_t)
ax[1].set_xlabel("T")
ax[1].set_ylabel("F(T)")
fig.tight_layout()

For your homework, you will recompute the necessary pollutant reductions based on the mean transfer coefficient if the mean, standard deviation, and lower bound of the distribution of $U_a$ are 1.2 m/s, 0.6 m/s and 0.01 m/s, respectively.

Then you will do it once more using chance constraints, assuming the air quality standard must be met at all receptors with a probability of 0.95.